### Inference of Energy Consumption to serverless functions
This script is dedicated to demonstrate the concept of distributing the energy consumption of a system to the serverless functions that are deployed to it.

To this end, the CPU Usage of the serverless function containers during time interval t is put into relation with the total CPU Usage of the system during time interval t and then multiplied with the measured energy consumption measured during time interval t.

In [1]:
# Imports
import requests
import logging
import json
import time

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s >>> %(message)s',
                        handlers=[logging.StreamHandler()])
logger = logging.getLogger(__name__)

In [2]:
# Data collection settings
DATA_COLLECTION_INTERVAL_SECONDS = 60

# Prometheus settings
PROMETHEUS_BASE_URL = "http://localhost:30009"
REST_API_PATH = "/api/v1/query"

In [3]:
def prom_instant_query(query_str: str) -> dict: 
    """ Executes a Prometheus API Query using the given query. """
    query_url = f"{PROMETHEUS_BASE_URL}/{REST_API_PATH}"
    response = requests.get(query_url, params={'query': query_str})
    
    if response:
        return response.json()
    else:
        logger.info(f"API Request ({response.url}) was not successful: HTTP {response.status_code}")
        return dict()

def get_fn_container_cpu_usages(namespace: str = "openfaas-fn"):
    """ Returns the CPU usage of all serverless function containers in the given namespace. """
    query = "rate(container_cpu_usage_seconds_total{image!='', namespace='openfaas-fn', container_name!='POD'}[1m]) > 0"
    query_result = prom_instant_query(query_str=query)
    
    if query_result:
        logger.info("FUNCTION CONTAINER CPU USAGES")
        logger.info(json.dumps(query_result, indent=4))
        return query_result
    else:
        logger.error(f"Failed to retrieve the CPU Usages of containers in namespace {namespace}.")
        return dict()
    
def get_container_cpu_usage_sum(container_name: str, node: str):
    """ Returns the sum of the CPU usage of all containers with the given container_name on the given
        node. """
    query = "sum(rate(container_cpu_usage_seconds_total{image!='', container='%s', container_name!='POD', node='%s'}[1m]))" % (container_name, node)
    query_result = prom_instant_query(query_str=query)
    
    if query_result:
        logger.info(f"SUM OF CPU USAGE OF CONTAINER {container_name.upper()} ON NODE {node.upper()}")
        logger.info(json.dumps(query_result, indent=4))
        timestamp, sum_cpu_usage = tuple(query_result['data']['result'][0]['value'])
        return sum_cpu_usage
    else:
        logger.error(f"Failed to retrieve the container CPU usage sum for '{container_name}' on node '{node}'")
        return dict()
    
def get_node_cpu_usage_sum(node: str):
    """ Returns the sum of the CPU usage of all containers on the given node. """
    query = "sum(rate(container_cpu_usage_seconds_total{image!='', container_name!='POD', node='%s'}[1m]))" % node
    query_result = prom_instant_query(query_str=query)
    
    if query_result:
        logger.info(f"NODE CPU USAGE SUM OF NODE {node.upper()}")
        logger.info(json.dumps(query_result, indent=4))
        timestamp, sum_cpu_usage = tuple(query_result['data']['result'][0]['value'])
        return sum_cpu_usage
    else:
        logger.error(f"Failed to retrieve the node cpu usage sum of node '{node}'")
        return dict()
    
def get_node_energy_consumption(node: str):
    """ Returns the energy consumption of the given node. """
    query = "idelta(powerexporter_power_consumption_ampere_seconds_total{instance='%s'}[2m:1m])" % node
    query_result = prom_instant_query(query_str=query)
    
    if query_result:
        logger.info(f"ENERGY CONSUMPTION OF NODE {node.upper()}")
        logger.info(json.dumps(query_result, indent=4))
        timestamp, energy_consumption = tuple(query_result['data']['result'][0]['value'])
        return energy_consumption
    else:
        logger.error(f"Failed to retrieve the energy consumption of node '{node}'")
        return dict()
    
def collect_data():
    while True:
        container_map = dict()
        energy_map = dict()
        # Step 1: Get CPU usage of all containers in openfaas-fn
        # This gives us two things in 1 query:
        #   1) the various serverless functions that were deployed in the time frame
        #   2) the deployment node per container
        fn_container_cpu_usages = get_fn_container_cpu_usages()
        for fn_container in fn_container_cpu_usages['data']['result']:
            container_name = fn_container['metric']['container']
            
            node = fn_container['metric']['node']
            # Step 2: Get Node CPU Usage as sum of cpu usage of all containers
            sum_node_cpu_usage = get_node_cpu_usage_sum(node)
            
            # Step 3: Get energy consumption of node
            if node not in energy_map:
                energy_consumption = get_node_energy_consumption(node)
                energy_map.update({node: float(energy_consumption)})
                
            cpu_usages_per_node = container_map.setdefault(container_name, dict()) # node -> cpu usage of fn containers
            if node not in cpu_usages_per_node:
                sum_cpu_usage_container = get_container_cpu_usage_sum(container_name, node)
                cpu_usages_per_node.update({node: float(sum_cpu_usage_container) / float(sum_node_cpu_usage)})
            
            logger.info(energy_map)
            logger.info(container_map)
            
            logger.info("==============================================================")
            # Print results:
            for node in energy_map:
                logger.info(f"Energy Consumption of {node} = {energy_map.get(node)}")
                
            for function, cpu_per_node in container_map.items():
                for node, cpu_usage in cpu_per_node.items():
                    logger.info(f"Energy Consumption ({function} | {node}) = {cpu_usage * energy_map.get(node)}")
            
        
        logger.info(f"Sleeping for {DATA_COLLECTION_INTERVAL_SECONDS} seconds ...")
        time.sleep(DATA_COLLECTION_INTERVAL_SECONDS)

In [ ]:
# Collect data and distribute energy consumption until CTRL + C
collect_data()

2022-08-02 18:05:47,473 - __main__ - INFO >>> FUNCTION CONTAINER CPU USAGES
2022-08-02 18:05:47,477 - __main__ - INFO >>> {
    "status": "success",
    "data": {
        "resultType": "vector",
        "result": [
            {
                "metric": {
                    "container": "analyze-sentence",
                    "cpu": "total",
                    "endpoint": "https-metrics",
                    "id": "/kubepods/besteffort/podf180d417-a88c-481b-878c-f0d798ff4518/cecc6fc7293a422f1834867c2b3c5f32d046332fa76d2ea2ec5080df9dd6278a",
                    "image": "docker.io/phyz1x/analyze-sentence:1.0.0",
                    "instance": "131.159.84.99:10250",
                    "job": "kubelet",
                    "metrics_path": "/metrics/cadvisor",
                    "name": "cecc6fc7293a422f1834867c2b3c5f32d046332fa76d2ea2ec5080df9dd6278a",
                    "namespace": "openfaas-fn",
                    "node": "odroidxu4-1",
                    "pod": "analyze-sente

2022-08-02 18:06:48,229 - __main__ - INFO >>> {
    "status": "success",
    "data": {
        "resultType": "vector",
        "result": [
            {
                "metric": {},
                "value": [
                    1659456406.928,
                    "0.09156813333331335"
                ]
            }
        ]
    }
}
2022-08-02 18:06:48,329 - __main__ - INFO >>> ENERGY CONSUMPTION OF NODE ODROIDXU4-1
2022-08-02 18:06:48,330 - __main__ - INFO >>> {
    "status": "success",
    "data": {
        "resultType": "vector",
        "result": [
            {
                "metric": {
                    "container": "power-exporter",
                    "endpoint": "http",
                    "instance": "odroidxu4-1",
                    "job": "power-exporter",
                    "namespace": "monitoring",
                    "pod": "power-exporter-4cpj8",
                    "service": "power-exporter"
                },
                "value": [
                    1

2022-08-02 18:07:49,334 - __main__ - INFO >>> SUM OF CPU USAGE OF CONTAINER NODEINFO ON NODE ODROIDXU4-1
2022-08-02 18:07:49,335 - __main__ - INFO >>> {
    "status": "success",
    "data": {
        "resultType": "vector",
        "result": [
            {
                "metric": {},
                "value": [
                    1659456468.03,
                    "0.0008888444444444587"
                ]
            }
        ]
    }
}
2022-08-02 18:07:49,336 - __main__ - INFO >>> {'odroidxu4-1': 32.27491204762373}
2022-08-02 18:07:49,336 - __main__ - INFO >>> {'analyze-sentence': {'odroidxu4-1': 0.9973013057704126}, 'nodeinfo': {'odroidxu4-1': 9.544043794748528e-05}}
2022-08-02 18:07:49,337 - __main__ - INFO >>> =================================================
2022-08-02 18:07:49,338 - __main__ - INFO >>> Energy Consumption of odroidxu4-1 = 32.27491204762373
2022-08-02 18:07:49,339 - __main__ - INFO >>> Energy Consumption (analyze-sentence | odroidxu4-1) = 32.187811928720365
2022

2022-08-02 18:09:50,207 - __main__ - INFO >>> NODE CPU USAGE SUM OF NODE ODROIDXU4-1
2022-08-02 18:09:50,208 - __main__ - INFO >>> {
    "status": "success",
    "data": {
        "resultType": "vector",
        "result": [
            {
                "metric": {},
                "value": [
                    1659456588.906,
                    "0.07212711111115151"
                ]
            }
        ]
    }
}
2022-08-02 18:09:50,307 - __main__ - INFO >>> ENERGY CONSUMPTION OF NODE ODROIDXU4-1
2022-08-02 18:09:50,308 - __main__ - INFO >>> {
    "status": "success",
    "data": {
        "resultType": "vector",
        "result": [
            {
                "metric": {
                    "container": "power-exporter",
                    "endpoint": "http",
                    "instance": "odroidxu4-1",
                    "job": "power-exporter",
                    "namespace": "monitoring",
                    "pod": "power-exporter-4cpj8",
                    "service":

2022-08-02 18:10:51,155 - __main__ - INFO >>> {
    "status": "success",
    "data": {
        "resultType": "vector",
        "result": [
            {
                "metric": {},
                "value": [
                    1659456649.85,
                    "0.07329593333334657"
                ]
            }
        ]
    }
}
2022-08-02 18:10:51,256 - __main__ - INFO >>> SUM OF CPU USAGE OF CONTAINER NODEINFO ON NODE ODROIDXU4-1
2022-08-02 18:10:51,257 - __main__ - INFO >>> {
    "status": "success",
    "data": {
        "resultType": "vector",
        "result": [
            {
                "metric": {},
                "value": [
                    1659456649.954,
                    "0.0032457555555555394"
                ]
            }
        ]
    }
}
2022-08-02 18:10:51,258 - __main__ - INFO >>> {'odroidxu4-1': 9.434512170611924}
2022-08-02 18:10:51,258 - __main__ - INFO >>> {'analyze-sentence': {'odroidxu4-1': 0.038377451288162806}, 'nodeinfo': {'odroidxu4-1': 0.0

...

2022-08-02 18:23:59,631 - __main__ - INFO >>> NODE CPU USAGE SUM OF NODE ODROIDXU4-1
2022-08-02 18:23:59,631 - __main__ - INFO >>> {
    "status": "success",
    "data": {
        "resultType": "vector",
        "result": [
            {
                "metric": {},
                "value": [
                    1659457438.317,
                    "1.074218689666676"
                ]
            }
        ]
    }
}
2022-08-02 18:23:59,730 - __main__ - INFO >>> ENERGY CONSUMPTION OF NODE ODROIDXU4-1
2022-08-02 18:23:59,731 - __main__ - INFO >>> {
    "status": "success",
    "data": {
        "resultType": "vector",
        "result": [
            {
                "metric": {
                    "container": "power-exporter",
                    "endpoint": "http",
                    "instance": "odroidxu4-1",
                    "job": "power-exporter",
                    "namespace": "monitoring",
                    "pod": "power-exporter-4cpj8",
                    "service": "

2022-08-02 18:25:00,673 - __main__ - INFO >>> SUM OF CPU USAGE OF CONTAINER NODEINFO ON NODE ODROIDXU4-1
2022-08-02 18:25:00,674 - __main__ - INFO >>> {
    "status": "success",
    "data": {
        "resultType": "vector",
        "result": [
            {
                "metric": {},
                "value": [
                    1659457499.36,
                    "0.13052188888888894"
                ]
            }
        ]
    }
}
2022-08-02 18:25:00,675 - __main__ - INFO >>> {'odroidxu4-1': 31.270527206719635}
2022-08-02 18:25:00,676 - __main__ - INFO >>> {'analyze-sentence': {'odroidxu4-1': 0.9567111433093357}, 'nodeinfo': {'odroidxu4-1': 0.034233099132331545}}
2022-08-02 18:25:00,677 - __main__ - INFO >>> =================================================
2022-08-02 18:25:00,677 - __main__ - INFO >>> Energy Consumption of odroidxu4-1 = 31.270527206719635
2022-08-02 18:25:00,678 - __main__ - INFO >>> Energy Consumption (analyze-sentence | odroidxu4-1) = 29.91686183582643
2022-0

2022-08-02 18:27:01,710 - __main__ - INFO >>> NODE CPU USAGE SUM OF NODE ODROIDXU4-1
2022-08-02 18:27:01,711 - __main__ - INFO >>> {
    "status": "success",
    "data": {
        "resultType": "vector",
        "result": [
            {
                "metric": {},
                "value": [
                    1659457620.395,
                    "2.426745044444463"
                ]
            }
        ]
    }
}
2022-08-02 18:27:01,810 - __main__ - INFO >>> ENERGY CONSUMPTION OF NODE ODROIDXU4-1
2022-08-02 18:27:01,811 - __main__ - INFO >>> {
    "status": "success",
    "data": {
        "resultType": "vector",
        "result": [
            {
                "metric": {
                    "container": "power-exporter",
                    "endpoint": "http",
                    "instance": "odroidxu4-1",
                    "job": "power-exporter",
                    "namespace": "monitoring",
                    "pod": "power-exporter-4cpj8",
                    "service": "

2022-08-02 18:28:02,742 - __main__ - INFO >>> SUM OF CPU USAGE OF CONTAINER NODEINFO ON NODE ODROIDXU4-1
2022-08-02 18:28:02,743 - __main__ - INFO >>> {
    "status": "success",
    "data": {
        "resultType": "vector",
        "result": [
            {
                "metric": {},
                "value": [
                    1659457681.426,
                    "0.003339044444444362"
                ]
            }
        ]
    }
}
2022-08-02 18:28:02,744 - __main__ - INFO >>> {'odroidxu4-1': 9.868255737064828}
2022-08-02 18:28:02,744 - __main__ - INFO >>> {'analyze-sentence': {'odroidxu4-1': 0.09508407273197667}, 'nodeinfo': {'odroidxu4-1': 0.05445402283229479}}
2022-08-02 18:28:02,745 - __main__ - INFO >>> =================================================
2022-08-02 18:28:02,746 - __main__ - INFO >>> Energy Consumption of odroidxu4-1 = 9.868255737064828
2022-08-02 18:28:02,746 - __main__ - INFO >>> Energy Consumption (analyze-sentence | odroidxu4-1) = 0.9383139462408181
2022-

2022-08-02 18:30:03,590 - __main__ - INFO >>> NODE CPU USAGE SUM OF NODE ODROIDXU4-1
2022-08-02 18:30:03,593 - __main__ - INFO >>> {
    "status": "success",
    "data": {
        "resultType": "vector",
        "result": [
            {
                "metric": {},
                "value": [
                    1659457802.271,
                    "0.06208131111113088"
                ]
            }
        ]
    }
}
2022-08-02 18:30:03,695 - __main__ - INFO >>> ENERGY CONSUMPTION OF NODE ODROIDXU4-1
2022-08-02 18:30:03,696 - __main__ - INFO >>> {
    "status": "success",
    "data": {
        "resultType": "vector",
        "result": [
            {
                "metric": {
                    "container": "power-exporter",
                    "endpoint": "http",
                    "instance": "odroidxu4-1",
                    "job": "power-exporter",
                    "namespace": "monitoring",
                    "pod": "power-exporter-4cpj8",
                    "service":